In [87]:
from typing import *

# template class to be implemented by specific child classes
class Player:
    def __init__(self):
        self.history: List[str] = []
        self.epoch_score = 0

    def get_first_action(self):
        raise NotImplementedError()

    def get_action(self, opponent_last_action: str):
        # for second action and onward
        raise NotImplementedError()
    
    def reset_state(self):
        # the states like history must be reset before starting a new game
        raise NotImplementedError()

# easy way to flip actions
def inverse(action: str):
    if action == "C":
        return "D"
    return "C"

In [88]:
# implementations of classic agents
# pretty naive implementations; lots of micro-optimizations possible
class ALLC(Player):
    def __init__(self):
        super().__init__()
    def get_first_action(self):
        return "C"
    def get_action(self, opponent_last_action: str):
        return "C"
    def reset_state(self):
        pass

class ALLD(Player):
    def __init__(self):
        super().__init__()
    def get_first_action(self):
        return "D"
    def get_action(self, opponent_last_action: str):
        return "D"
    def reset_state(self):
        pass
    
class TFT(Player):
    def __init__(self):
        super().__init__()
    def get_first_action(self):
        return "C"
    def get_action(self, opponent_last_action: str):
        return opponent_last_action
    def reset_state(self):
        pass

class HANDSHAKE(Player):
    def __init__(self, initial_actions: List[str]=["C", "D"]):
        super().__init__()
        self.initial_actions = initial_actions
        self.ALLD_mode = False # becomes True if the opponent fails the initial handshake
    def get_first_action(self):
        return self.initial_actions[0]
    def get_action(self, opponent_last_action: str):
        if self.ALLD_mode:
            return "D"
        # if it hasn't finished the handshake yet...
        if len(self.history) < len(self.initial_actions):
            self.history.append(opponent_last_action)
            num_rounds_played = len(self.history)
            if opponent_last_action != self.initial_actions[num_rounds_played-1]:
                # handshake failed; always defect from now on
                self.ALLD_mode = True
                return "D"
            # otherwise, play the next planned action
            if num_rounds_played < len(self.initial_actions):
                return self.initial_actions[num_rounds_played]
            # if right after successful handshake, play "C" always -- do nothing here
        
        # ALLC
        return "C"
    def reset_state(self):
        self.history = []
        self.ALLD_mode = False

class SLAVE(Player):
    # can defect even within the initial actions; otherwise as defined in Li 2018
    def __init__(self, initial_actions: List[str]=["C", "D", "C", "C", "D"]):
        super().__init__()
        self.initial_actions = initial_actions
        self.ALLD_mode = False # becomes True if the opponent fails the initial handshake
    def get_first_action(self):
        return self.initial_actions[0]
    def get_action(self, opponent_last_action: str):
        if self.ALLD_mode:
            return "D"
        # if it hasn't finished the handshake yet...
        if len(self.history) < len(self.initial_actions):
            self.history.append(opponent_last_action)
            num_rounds_played = len(self.history)
            if opponent_last_action != self.initial_actions[num_rounds_played-1]:
                # handshake failed; always defect from now on
                self.ALLD_mode = True
                return "D"
            # otherwise, play the next planned action
            if num_rounds_played < len(self.initial_actions):
                return self.initial_actions[num_rounds_played]
            else:
                # first play of a reverse TFT
                return "D"
        
        # reverse TFT
        return inverse(opponent_last_action)
    def reset_state(self):
        self.history = []
        self.ALLD_mode = False

class MASTER(Player):
    # can defect even within the initial actions; otherwise as defined in Li 2021
    # "Collective Strategies with a Master-slave Mechanism Dominate in Spatial Iterated Prisoner’s Dilemma"
    def __init__(self, initial_actions: List[str]=["C", "D", "C", "C", "D"]):
        super().__init__()
        self.initial_actions = initial_actions
        self.ALLD_mode = False # becomes True if the opponent fails the initial handshake
    def get_first_action(self):
        return self.initial_actions[0]
    def get_action(self, opponent_last_action: str):
        if self.ALLD_mode:
            return "D"
        # if it hasn't finished the handshake yet...
        if len(self.history) < len(self.initial_actions):
            self.history.append(opponent_last_action)
            num_rounds_played = len(self.history)
            if opponent_last_action != self.initial_actions[num_rounds_played-1]:
                # handshake failed; always defect from now on
                self.ALLD_mode = True
                return "D"
            # otherwise, play the next planned action
            if num_rounds_played < len(self.initial_actions):
                return self.initial_actions[num_rounds_played]
            else:
                # first play of a GRUDGER
                return "C"
        
        # GRUDGER
        if opponent_last_action == "D":
            self.ALLD_mode = True
            return "D"
        return "C"
    def reset_state(self):
        self.history = []
        self.ALLD_mode = False


In [89]:
import random

# TODO: add round discount to simulate truly unknown number of rounds?

# 1v1 game to test the implementations
def get_payoffs(action_A: str, action_B: str) -> tuple[int, int]:
    if action_A == "C":
        if action_B == "C":
            return 3, 3 # C, C
        return 0, 5 # C, D
    if action_B == "C":
        return 5, 0 # D, C
    return 1, 1 # D, D

def one_game(player_A: Player, player_B: Player, num_rounds: int, noise_level: float=0):
    # get the first actions, possibly affected by noise
    last_action_A: str = player_A.get_first_action()
    last_action_B: str = player_B.get_first_action()
    if random.random() < noise_level:
        last_action_A = inverse(last_action_A)
    if random.random() < noise_level:
        last_action_B = inverse(last_action_B)

    payoff_A, payoff_B = get_payoffs(last_action_A, last_action_B)
    score_A, score_B = payoff_A, payoff_B
    # print(last_action_A, last_action_B, payoff_A, payoff_B, score_A, score_B) # for testing
    
    for _ in range(1, num_rounds):
        # get the next actions, possibly affected by noise
        last_action_A_cache = last_action_A
        last_action_A: str = player_A.get_action(last_action_B)
        last_action_B: str = player_B.get_action(last_action_A_cache)
        if random.random() < noise_level:
            last_action_A = inverse(last_action_A)
        if random.random() < noise_level:
            last_action_B = inverse(last_action_B)
        
        payoff_A, payoff_B = get_payoffs(last_action_A, last_action_B)
        score_A += payoff_A
        score_B += payoff_B
        # print(last_action_A, last_action_B, payoff_A, payoff_B, score_A, score_B) # for testing
    
    # IMPORTANT: reset players' state!
    player_A.reset_state()
    player_B.reset_state()
    return score_A, score_B


In [90]:
# testing the one epoch evaluation code
# turn them into assertions instead?
# print(one_game(ALLC(), ALLC(), 10, 0))
# print(one_game(ALLC(), ALLD(), 10, 0))
# print(one_game(ALLC(), TFT(), 10, 0))
# print(one_game(ALLC(), HANDSHAKE(), 10, 0))
# print(one_game(ALLC(), MASTER(), 10, 0))
# print(one_game(ALLC(), SLAVE(), 10, 0))
# print(one_game(ALLD(), ALLD(), 10, 0))
# print(one_game(ALLD(), TFT(), 10, 0))
# print(one_game(ALLD(), HANDSHAKE(), 10, 0))
# print(one_game(ALLD(), MASTER(), 10, 0))
# print(one_game(ALLD(), SLAVE(), 10, 0))
# print(one_game(TFT(), TFT(), 10, 0))
# print(one_game(TFT(), HANDSHAKE(), 10, 0))
# print(one_game(TFT(), MASTER(), 10, 0))
# print(one_game(TFT(), SLAVE(), 10, 0))
# print(one_game(HANDSHAKE(), HANDSHAKE(), 10, 0))
# print(one_game(HANDSHAKE(), MASTER(), 10, 0))
# print(one_game(HANDSHAKE(), SLAVE(), 10, 0))
# print(one_game(MASTER(), MASTER(), 10, 0))
# print(one_game(MASTER(), SLAVE(), 10, 0))
# print(one_game(SLAVE(), SLAVE(), 10, 0))

# Noise?
# print(one_game(TFT(), TFT(), 10, 0.05))

In [91]:
# define Chameleon
class CHAMELEON(Player):
    def __init__(self, master_initial_actions: List[str] = ["C", "D", "C", "C", "D"]):
        super().__init__()
        self.ALLD_mode = False # becomes True if the opponent fails the initial handshake
        self.MASTER_mode = False # becomes True if the opponent played CD first
        self.master_initial_actions = master_initial_actions
    def get_first_action(self):
        return "C"
    def get_action(self, opponent_last_action: str):
        if self.ALLD_mode:
            return "D"
        
        if self.MASTER_mode:
            # if it hasn't finished the handshake yet...
            if len(self.history) < len(self.master_initial_actions):
                self.history.append(opponent_last_action)
                num_rounds_played = len(self.history)
                if opponent_last_action != self.master_initial_actions[num_rounds_played-1]:
                    # handshake failed; always defect from now on
                    self.ALLD_mode = True
                    return "D"
                # otherwise, play the next planned action
                if num_rounds_played < len(self.master_initial_actions):
                    return self.master_initial_actions[num_rounds_played]
                else:
                    # first play of a GRUDGER
                    return "C"
            
            # GRUDGER
            if opponent_last_action == "D":
                self.ALLD_mode = True
                return "D"
            return "C"

        if len(self.history) < 1:
            # this is the second action
            self.history.append(opponent_last_action)
            if opponent_last_action == "D":
                self.ALLD_mode = True # opponent must be ALLD
            return "D" # try "D" regardless

        if len(self.history) < 2:
            self.history.append(opponent_last_action)
            if opponent_last_action == "C":
                # opponent played CC for their first 2 actions
                return "D"
            # else, opponent played CD for their first 2 actions
            # act like a MASTER from now on
            self.MASTER_mode = True
            return "C"

        if len(self.history) < 3:
            self.history.append(opponent_last_action)
            if opponent_last_action == "C":
                # opponent played CCC for their first 3 actions; must be ALLC
                self.ALLD_mode = True
                return "D"

            # else, opponent played CCD for their first 3 actions; must be TFT
            # then, act like an ALLC from now on (for forgiveness) -- do nothing here
        
        # ALLC
        return "C"
        # we COULD mimic a TFT to be more robust to unknown
        # algorithms, and more robust to noise
        
    def reset_state(self):
        self.history = []
        self.ALLD_mode = False
        self.MASTER_mode = False
    
        

In [92]:
# assert one_game(CHAMELEON(), CHAMELEON(), 10, 0) == (26, 26)
# assert one_game(CHAMELEON(), ALLC(), 10, 0) == (48, 3)
# assert one_game(CHAMELEON(), ALLD(), 10, 0) == (9, 14)
# assert one_game(CHAMELEON(), TFT(), 10, 0) == (27, 27)
# assert one_game(CHAMELEON(), HANDSHAKE(), 10, 0) == (40, 10)
# assert one_game(CHAMELEON(), MASTER(), 10, 0) == (26, 26)
# assert one_game(CHAMELEON(), SLAVE(), 10, 0) == (27, 17)

In [93]:
class EIPD:
    # Evolutionary Iterated Prisoner's Dilemma
    def __init__(self, initial_players: List[Player]=[], num_rounds: int=200, num_replacements: int=5, noise_level: float=0):
        self.num_players = len(initial_players)
        self.players = initial_players
        self.num_rounds = num_rounds
        self.num_replacements = num_replacements
        self.noise_level = noise_level
    
    def one_epoch(self):
        # round robin matchmaking
        for i in range(self.num_players):
            for j in range(i+1, self.num_players):
                player_i, player_j = self.players[i], self.players[j]
                score_i, score_j = one_game(player_i, player_j, self.num_rounds, self.noise_level)
                player_i.epoch_score += score_i
                player_j.epoch_score += score_j
                # for testing:
                # print(type(player_i).__name__, type(player_j).__name__, score_i, score_j, player_i.epoch_score, player_j.epoch_score)
        
        # for testing: print everyone's type and score
        # for i in range(self.num_players):
        #     player_i = self.players[i]
        #     print(f'{type(player_i).__name__}, score: {player_i.epoch_score}')
        
        sorted_players = sorted(self.players, key=lambda player: player.epoch_score)

        # for testing: print everyone's type and score after sorting
        # for i in range(self.num_players):
        #     player_i = sorted_players[i]
        #     print(f'{type(player_i).__name__}, score: {player_i.epoch_score}')

        # evolution: replace the lowest performing players with the highest performing ones
        # TODO: randomize the reproduction (necessary for when
        # players of multiple species tie in score)
        new_players = []
        for i in range(1, self.num_replacements+1):
            reproduced_player = sorted_players[-i]
            new_players.append(type(reproduced_player)())
        for i in range(self.num_replacements, self.num_players):
            remained_player = sorted_players[i]
            new_players.append(type(remained_player)())
        
        self.players = new_players
        
        # for testing: print the composition of new players
        new_player_species = []
        for i in range(self.num_players):
            new_player_species.append(type(self.players[i]).__name__)
        print(Counter(new_player_species))


In [122]:
# play around with different initial compositions here!
ALLC_COUNT = 3
ALLD_COUNT = 3
TFT_COUNT = 40
HANDSHAKE_COUNT = 2
MASTER_COUNT = 5
SLAVE_COUNT = 5
CHAMELEON_COUNT = 0

# ALLC_COUNT = 6
# ALLD_COUNT = 10
# TFT_COUNT = 6
# HANDSHAKE_COUNT = 6
# MASTER_COUNT = 12
# SLAVE_COUNT = 12
# CHAMELEON_COUNT = 1

# ALLC_COUNT = 0
# ALLD_COUNT = 22
# TFT_COUNT = 3
# HANDSHAKE_COUNT = 0
# MASTER_COUNT = 0
# SLAVE_COUNT = 0
# CHAMELEON_COUNT = 0

players = []
for _ in range(ALLC_COUNT): players.append(ALLC())
for _ in range(ALLD_COUNT): players.append(ALLD())
for _ in range(TFT_COUNT): players.append(TFT())
for _ in range(HANDSHAKE_COUNT): players.append(HANDSHAKE())
for _ in range(MASTER_COUNT): players.append(MASTER())
for _ in range(SLAVE_COUNT): players.append(SLAVE())
for _ in range(CHAMELEON_COUNT): players.append(CHAMELEON())
eipd = EIPD(players, num_rounds=200)
for _ in range(15):
    eipd.one_epoch()

Counter({'TFT': 45, 'SLAVE': 5, 'MASTER': 5, 'ALLC': 3})
Counter({'TFT': 50, 'MASTER': 5, 'ALLC': 3})
Counter({'TFT': 55, 'ALLC': 3})
Counter({'TFT': 55, 'ALLC': 3})
Counter({'TFT': 55, 'ALLC': 3})
Counter({'TFT': 55, 'ALLC': 3})
Counter({'TFT': 55, 'ALLC': 3})
Counter({'TFT': 55, 'ALLC': 3})
Counter({'TFT': 55, 'ALLC': 3})
Counter({'TFT': 55, 'ALLC': 3})
Counter({'TFT': 55, 'ALLC': 3})
Counter({'TFT': 55, 'ALLC': 3})
Counter({'TFT': 55, 'ALLC': 3})
Counter({'TFT': 55, 'ALLC': 3})
Counter({'TFT': 55, 'ALLC': 3})


In [95]:
# run it 30 times with noise:
eipd = EIPD(players, num_rounds=200, noise_level=0.05)
for _ in range(30):
    eipd.one_epoch()

Counter({'CHAMELEON': 4, 'TFT': 2})
Counter({'TFT': 3, 'CHAMELEON': 3})
Counter({'TFT': 4, 'CHAMELEON': 2})
Counter({'TFT': 5, 'CHAMELEON': 1})
Counter({'TFT': 5, 'CHAMELEON': 1})
Counter({'TFT': 5, 'CHAMELEON': 1})
Counter({'TFT': 4, 'CHAMELEON': 2})
Counter({'TFT': 4, 'CHAMELEON': 2})
Counter({'TFT': 4, 'CHAMELEON': 2})
Counter({'TFT': 4, 'CHAMELEON': 2})
Counter({'TFT': 5, 'CHAMELEON': 1})
Counter({'TFT': 5, 'CHAMELEON': 1})
Counter({'TFT': 5, 'CHAMELEON': 1})
Counter({'TFT': 4, 'CHAMELEON': 2})
Counter({'TFT': 5, 'CHAMELEON': 1})
Counter({'TFT': 4, 'CHAMELEON': 2})
Counter({'TFT': 5, 'CHAMELEON': 1})
Counter({'TFT': 5, 'CHAMELEON': 1})
Counter({'TFT': 5, 'CHAMELEON': 1})
Counter({'TFT': 4, 'CHAMELEON': 2})
Counter({'TFT': 5, 'CHAMELEON': 1})
Counter({'TFT': 5, 'CHAMELEON': 1})
Counter({'TFT': 4, 'CHAMELEON': 2})
Counter({'TFT': 5, 'CHAMELEON': 1})
Counter({'TFT': 5, 'CHAMELEON': 1})
Counter({'TFT': 5, 'CHAMELEON': 1})
Counter({'TFT': 4, 'CHAMELEON': 2})
Counter({'TFT': 4, 'CHAMELEO

In [96]:
# verify the claim that CHAMELEON performs the same as CSMSM
# against all other strategies, while outperform MASTER against TFT
assert one_game(CHAMELEON(), CHAMELEON(), 20) == one_game(MASTER(), CHAMELEON(), 20)
assert one_game(CHAMELEON(), ALLC(), 20) == one_game(MASTER(), ALLC(), 20)
assert one_game(CHAMELEON(), ALLD(), 20) == one_game(MASTER(), ALLD(), 20)
assert one_game(CHAMELEON(), HANDSHAKE(), 20) == one_game(MASTER(), HANDSHAKE(), 20)
assert one_game(CHAMELEON(), MASTER(), 20) == one_game(MASTER(), MASTER(), 20)
assert one_game(CHAMELEON(), SLAVE(), 20) == one_game(MASTER(), SLAVE(), 20)

print(f'CHAMELEON vs TFT: {one_game(CHAMELEON(), TFT(), 20)}')
print(f'MASTER vs TFT: {one_game(MASTER(), TFT(), 20)}')

CHAMELEON vs TFT: (57, 57)
MASTER vs TFT: (26, 21)


In [144]:
# for Demo in paper
players = []
for _ in range(6): players.append(ALLC())
for _ in range(6): players.append(ALLD())
for _ in range(15): players.append(TFT())
for _ in range(6): players.append(HANDSHAKE())
for _ in range(3): players.append(MASTER())
for _ in range(3): players.append(SLAVE())
for _ in range(1): players.append(CHAMELEON())
eipd = EIPD(players, num_rounds=200)

initial_composition = []
for i in range(len(players)): initial_composition.append(type(players[i]).__name__)
print(Counter(initial_composition))

for _ in range(15):
    eipd.one_epoch()

Counter({'TFT': 15, 'ALLC': 6, 'ALLD': 6, 'HANDSHAKE': 6, 'MASTER': 3, 'SLAVE': 3, 'CHAMELEON': 1})
Counter({'TFT': 15, 'MASTER': 6, 'ALLD': 6, 'HANDSHAKE': 6, 'SLAVE': 4, 'CHAMELEON': 2, 'ALLC': 1})
Counter({'TFT': 15, 'MASTER': 9, 'ALLD': 6, 'CHAMELEON': 4, 'SLAVE': 4, 'HANDSHAKE': 1, 'ALLC': 1})
Counter({'TFT': 15, 'MASTER': 10, 'CHAMELEON': 8, 'ALLD': 6, 'ALLC': 1})
Counter({'TFT': 15, 'CHAMELEON': 13, 'MASTER': 10, 'ALLD': 1, 'ALLC': 1})
Counter({'CHAMELEON': 18, 'TFT': 15, 'MASTER': 7})
Counter({'CHAMELEON': 23, 'TFT': 15, 'MASTER': 2})
Counter({'CHAMELEON': 28, 'TFT': 12})
Counter({'CHAMELEON': 23, 'TFT': 17})
Counter({'TFT': 22, 'CHAMELEON': 18})
Counter({'TFT': 27, 'CHAMELEON': 13})
Counter({'TFT': 32, 'CHAMELEON': 8})
Counter({'TFT': 37, 'CHAMELEON': 3})
Counter({'TFT': 40})
Counter({'TFT': 40})
Counter({'TFT': 40})
